In [1]:
!curl localhost:9200

{
  "name" : "3972dd1ec50e",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "mNZ2AP9wSA-yi_w7-_C8Tg",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        

In [3]:
set([doc['course'] for doc in documents])

{'data-engineering-zoomcamp', 'machine-learning-zoomcamp', 'mlops-zoomcamp'}

In [4]:
#!conda install conda-forge::elasticsearch=8.17.2 -y

In [5]:
from elasticsearch import Elasticsearch
from  tqdm import tqdm

In [6]:
es_client = Elasticsearch('http://localhost:9200')

In [7]:
es_client.info()

ObjectApiResponse({'name': '3972dd1ec50e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'mNZ2AP9wSA-yi_w7-_C8Tg', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [8]:
index_setting={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name='course-faq'

es_client.indices.create(index=index_name, body=index_setting)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-faq'})

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)


In [15]:

query="How do execute a command on a Kubernetes pod?"

#{'data-engineering-zoomcamp', 'machine-learning-zoomcamp', 'mlops-zoomcamp'}

search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            #"filter": {
            #    "term": {
            #        "course": "data-engineering-zoomcamp"
            #    }
            #}
        }
    }
}

es_client.search(index=index_name, body=search_query)

ObjectApiResponse({'took': 15, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-faq', '_id': '4oTOMJcBbltKjJAI-cu-', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-faq', '_id': 'cYTOMJcBbltKjJAI_MyN', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboa

In [16]:
es_client.get(index=index_name,id='hoTOMJcBbltKjJAI6skG')

ObjectApiResponse({'_index': 'course-faq', '_id': 'hoTOMJcBbltKjJAI6skG', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}})

In [21]:
query="How do copy a file to a Docker container?"

#{'data-engineering-zoomcamp', 'machine-learning-zoomcamp', 'mlops-zoomcamp'}

search_query={
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [31]:
records = [ rec['_source'] for rec in  response['hits']['hits']]
records

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [87]:
from functools import reduce

context_template = """
Q: {question}
A: {text}
""".strip()

context=reduce(lambda s1,s2: s1+"\n\n\n"+s2, [context_template.format(question=rec['question'],
                                                                     text=rec['text'])  for rec in records]).strip()
print(context)
print(f'context length: {len(context)}')

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani


Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how t

In [88]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt=prompt_template.format(question="How do I execute a command in a running docker container?",
                  context=context)

print(prompt)
print(f'prompt length: {len(prompt)}')

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id

In [85]:
#!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.1 MB/s eta 0:00:00a 0:00:01


In [90]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens=encoding.encode(prompt)
print(f'number of tokens: {len(tokens)}')

number of tokens: 322


In [104]:
with open('/home/mahdi/openrouter.key','r') as f_key:
    openrouter_key=f_key.read().strip()

In [119]:

import requests
import json

response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {openrouter_key}",
    "Content-Type": "application/json",
  },
  data=json.dumps({
    "model": "deepseek/deepseek-r1-0528:free",
    "messages": [
      {
        "role": "user",
        "content": prompt#"What is the meaning of life?"
      }
    ],
    
  })
)



In [120]:
print(response.json()['choices'][0]['message']['content'])

Based entirely on the provided context, here's how to execute a command in a running Docker container:

1.  **Find the Container ID:** Use `docker ps` to list running containers and identify the ID of the container you want to interact with.
2.  **Execute the Command:** Use the `docker exec -it` command followed by the container ID and the command you want to run (typically `bash` to get an interactive shell).
    ```
    docker exec -it <container-id> bash
    ```

**Source Context:** The answer is directly taken from the FAQ entry "How do I debug a docker container?" provided by Marcos MJD:
> "If the container is already running, execute a command in the specific container:
> docker ps (find the container-id)
> docker exec -it <container-id> bash"
